In [ ]:
# Supress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Python
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# NumPy and PyTorch
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader

# Custom
from path_loader import PathDataLoader
from networks import SiameseNetworkSimple
from losses import ContrastiveLossSimple
from patch_generator import PatchGenerator

# Set random seeds
np.random.seed(0)

In [ ]:
def init_data(paths, batch_size, ratio):

    all_data = []
    all_hash = []

    for i in range(len(paths)):
      for j in range(len(paths[0])):
        if len(paths[i][j]) != 0:
          for k in paths[i][j]:
            arr = []
            c = 0
            for l in range(len(k.points)):
              c += 1
              for m in range(len(k.points[l])):
                arr.append(k.points[l][m])
            for l in range((5-c)*3, 0, -1):
              arr.append(0)
            for l in range(len(k.interaction_types)):
              arr.append(k.interaction_types[l])
            for l in range(5-c, 0, -1):
              arr.append(0)
            all_hash.append(k.hash)
            arr.append(k.path_gain_db)
            min_val = min(arr)
            max_val = max(arr)

            # Normalize the list to the range -1 to 1
            normalized_data = [2 * (x - min_val) / (max_val - min_val) - 1 for x in arr]
            all_data.append(normalized_data)

    data_pairs = []
    tol = 1200000
    for i in range(len(all_data)):
        for j in range(len(all_data)):
            if i != j:  # Ensure the pairs are not identical
                if abs(all_hash[i] - all_hash[j]) < tol:
                  label = 0
                else:
                  label = 1
                data_pairs.append((torch.tensor(all_data[i], dtype=torch.float), torch.tensor(all_data[j], dtype=torch.float), torch.tensor(label, dtype=torch.long)))
        print(len(data_pairs))
        if len(data_pairs) > 100000:
          break

    dataloaders = DataLoader(data_pairs, batch_size, shuffle=True, drop_last=True)
    """
    numpy_data = np.array(data_pairs)

    datasets = {}
    datasets['train'] = np.array(numpy_data[:int(len(numpy_data) * ratio)])
    datasets['val'] = np.array(numpy_data[int(len(numpy_data) * ratio):])


    # Shuffle data
    dataset_length = len(datasets['train'])
    indices = list(range(dataset_length))
    np.random.shuffle(indices)
    sampler = torch.utils.data.SubsetRandomSampler(indices)

    # Create DataLoaders using these samplers
    dataloaders = {
        'train': torch.utils.data.DataLoader(
            datasets['train'],
            batch_size=batch_size,
            sampler=sampler,
            num_workers=2,
            pin_memory=False,
        ),
        'val': torch.utils.data.DataLoader(
            datasets['val'],
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=False,
        )
    }
    """

    return dataloaders

In [ ]:
def train_siamese_network(train_loader, net, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        for i, data in enumerate(train_loader, 0):
            input1, input2, label = data
            output1, output2 = net(input1), net(input2)
            loss = criterion(output1, output2, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 1000 == 0:
                print(f"Epoch {epoch}, Iteration {i}, Loss {loss.item()}")

In [ ]:
def transform_patches(patches):
    for i in range(len(patches)):
        for j in range(len(patches[i])):
            path = patches[i][j]
            arr = []
            c = 0
            for l in range(len(path.points)):
                c += 1
                for m in range(len(path.points[l])):
                    arr.append(path.points[l][m])
            for l in range((5-c)*3, 0, -1):
                arr.append(0)
            for l in range(len(path.interaction_types)):
                arr.append(path.interaction_types[l])
            for l in range(5-c, 0, -1):
                arr.append(0)
            arr.append(path.path_gain_db)
            min_val = min(arr)
            max_val = max(arr)

            # Normalize the list to the range -1 to 1
            normalized_data = [2 * (x - min_val) / (max_val - min_val) - 1 for x in arr]

            # Update list
            patches[i][j] = normalized_data
            
    return patches

In [9]:
# Load data
pathLoader = PathDataLoader()
paths = pathLoader.read('eu_city_2x2_macro_306.bin')[:10000]

In [ ]:
# Generate patches
gen = PatchGenerator(num_patches=8, attribute="transmitter")
patches = gen.generate_patches(paths)

# Transform PathPropagation objects to normalized feature vectors
patches = gen.transform_patches(patches)

In [ ]:
# Generate pairs

In [ ]:
# Format data
batch_size = 10
train_val_ratio = 0.95
dataloaders = init_data(paths, batch_size, train_val_ratio)

In [ ]:
# Instantiate the Siamese Network and Loss Function
net = SiameseNetworkSimple()
criterion = ContrastiveLossSimple()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [ ]:
train_siamese_network(dataloaders, net, criterion, optimizer, epochs=2)

In [ ]:
# Convert embeddings to a list
embeddings_list = embeddings.squeeze().tolist()
embeddings_list2 = embeddings2.squeeze().tolist()
print(embeddings_list)

# Create x-axis indices
indices = list(range(len(embeddings_list)))

# Plot the 1D embeddings
plt.figure(figsize=(8, 6))
plt.plot(indices, embeddings_list, marker='o', linestyle='-')
plt.title('Visualization of 1D Embeddings')
plt.xlabel('Component')
plt.ylabel('Embedding Value')
plt.show()

# Plot the 1D embeddings
plt.figure(figsize=(8, 6))
plt.plot(indices, embeddings_list2, marker='o', linestyle='-')
plt.title('Visualization of 1D Embeddings')
plt.xlabel('Component')
plt.ylabel('Embedding Value')
plt.show()